# Environment and Dataframe Setup

##Mount Google Drive
Remember to add a shortcut to "COVID DGNN KG" in your own drive before mounting.

In [11]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


## Zero-fill the FIPS column of counties dataframe

In [12]:
import ast
import csv
import json
import numpy as np
from os import listdir, makedirs
from os.path import isfile, join, exists
import pandas as pd
from tqdm import tqdm

county_df = pd.read_csv("/content/gdrive/My Drive/COVID_DGNN_KG/counties_static_features.csv")
county_df['FIPS'] = county_df['FIPS'].astype(str)
county_df['FIPS'] = county_df['FIPS'].str.zfill(5)
county_df

,Unnamed: 0,FIPS,State,County,Life Expectancy,Age-Adjusted Death Rate,Child Mortality Rate,% Frequent Physical Distress,% Frequent Mental Distress,% Adults with Diabetes,HIV Prevalence Rate,% Food Insecure,% Limited Access to Healthy Foods,Drug Overdose Mortality Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,High School Graduation Rate,Average Grade Performance,Median Household Income,% Enrolled in Free or Reduced Lunch,Homicide Rate,Suicide Rate (Age-Adjusted),Firearm Fatalities Rate,% Homeowners,% Broadband Access,% Black,% American Indian & Alaska Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Rural,POPESTIMATE,POPEST_MALE,POPEST_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,...,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,0,01001,Alabama,Autauga County,77.2,409.0,57.0,13,16,13,230.0,16,12.0,8.0,20.0,38,89.0,3.1,58233.0,47.0,5.0,18.0,17.0,73,81,19.9,0.5,1.2,0.1,3.0,73.8,1,42.0,56145.0,27226.0,28919.0,3370.0,1746.0,1624.0,3488.0,...,1919.0,3666.0,1758.0,1908.0,3773.0,1840.0,1933.0,3595.0,1706.0,1889.0,3792.0,1827.0,1965.0,3646.0,1796.0,1850.0,3910.0,1920.0,1990.0,3457.0,1594.0,1863.0,2806.0,1360.0,1446.0,2406.0,1035.0,1371.0,1803.0,751.0,1052.0,1140.0,496.0,644.0,927.0,353.0,574.0,39.2,37.8,40.5
1,1,01003,Alabama,Baldwin County,78.2,366.0,51.0,12,15,10,169.0,13,5.0,13.0,17.0,36,88.0,3.2,59871.0,40.0,3.0,19.0,14.0,75,82,8.6,0.8,1.1,0.1,4.7,83.2,1,42.3,229287.0,111000.0,118287.0,12092.0,6317.0,5775.0,13409.0,...,6213.0,12962.0,6294.0,6668.0,13565.0,6590.0,6975.0,13920.0,6681.0,7239.0,14298.0,6948.0,7350.0,14610.0,6995.0,7615.0,16450.0,7758.0,8692.0,16997.0,7811.0,9186.0,15731.0,7232.0,8499.0,13973.0,6589.0,7384.0,9554.0,4560.0,4994.0,5678.0,2572.0,3106.0,4549.0,1878.0,2671.0,43.9,42.6,45.3
2,2,01005,Alabama,Barbour County,74.1,553.0,76.0,18,19,18,471.0,22,11.0,NaN,26.0,41,82.0,2.4,35972.0,61.0,10.0,11.0,21.0,61,61,47.8,0.7,0.5,0.2,4.5,45.5,2,67.8,24589.0,12987.0,11602.0,1316.0,674.0,642.0,1328.0,...,648.0,1640.0,998.0,642.0,1562.0,898.0,664.0,1496.0,915.0,581.0,1540.0,861.0,679.0,1457.0,795.0,662.0,1644.0,822.0,822.0,1496.0,733.0,763.0,1490.0,685.0,805.0,1412.0,620.0,792.0,982.0,446.0,536.0,582.0,244.0,338.0,516.0,174.0,342.0,40.9,38.9,44.4
3,3,01007,Alabama,Bibb County,73.4,595.0,103.0,15,17,14,235.0,15,3.0,22.0,27.0,40,88.0,2.7,47918.0,63.0,10.0,17.0,20.0,74,69,21.1,0.5,0.2,0.1,2.8,74.4,0,68.4,22136.0,11773.0,10363.0,1223.0,593.0,630.0,1193.0,...,702.0,1604.0,965.0,639.0,1507.0,884.0,623.0,1393.0,791.0,602.0,1601.0,879.0,722.0,1482.0,794.0,688.0,1551.0,808.0,743.0,1362.0,683.0,679.0,1163.0,546.0,617.0,1054.0,496.0,558.0,697.0,305.0,392.0,468.0,190.0,278.0,398.0,138.0,260.0,40.3,38.4,43.0
4,4,01009,Alabama,Blount County,74.4,520.0,82.0,15,17,15,91.0,14,3.0,22.0,29.0,39,93.0,2.9,52902.0,50.0,7.0,20.0,21.0,79,73,1.5,0.6,0.3,0.1,9.7,86.8,2,90.0,57879.0,28617.0,29262.0,3437.0,1792.0,1645.0,3590.0,...,1740.0,3428.0,1698.0,1730.0,3424.0,1700.0,1724.0,3525.0,1761.0,1764.0,3747.0,1879.0,1868.0,3867.0,1930.0,1937.0,4018.0,2010.0,2008.0,3690.0,1835.0,1855.0,3340.0,1616.0,1724.0,3072.0,1416.0,1656.0,2158.0,964.0,1194.0,1318.0,590.0,728.0,1111.0,409.0,702.0,41.2,40.3,42.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,3137,

## Construct Input and Output Files



In [13]:
year_months = ["2021-08", "2021-09", "2021-10"]
input_folder = "/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/"
output_folder = "/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_processed/"

filenames_in = []
for year_month in year_months:
  for i in range(1, 5):
    input_filename = "patterns-" + year_month + "-part" + str(i) + ".csv.gz"
    path = join(input_folder, input_filename)
    if not exists(path):
      print(path + " not found")
    else:
      filenames_in.append(path)

filenames_in

['/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part1.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part2.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part3.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part4.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part1.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part2.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part3.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part4.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part1.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part2.csv.gz',
 '/content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part3.csv.gz',

#Data Aggregation

In [16]:
import csv
def aggregate_data(filename_in):
  #filename_in = filenames_in[0]
  print("aggregate for file: " + filename_in)
  # Extract only the origin and destination CBG columns of the original dataset
  main_df = pd.read_csv(filename_in, compression='gzip', usecols=['poi_cbg', 'visitor_home_aggregation'])

  # create a list of datasets from the extracted dataset 
  n = 1000 #chunk row size
  chunks_list = [main_df[i : i + n] for i in range(0,main_df.shape[0], n)]

  # Extract each individual home CBG and number of visitors from the visitor_home_cbgs column
  # Loop over each dataset of 1000 rows and aggregate the resulting datasets
  parsed_df = []
  for sample_df in tqdm(chunks_list):
    sample_df = chunks_list[0]
    long_sample = (pd.DataFrame(sample_df['visitor_home_aggregation'].apply(ast.literal_eval).values.tolist(), index=sample_df['poi_cbg'])
    .stack()
    .reset_index()
    .rename(columns={'level_1':'visitor_home_tract', 0:'number_of_visitors'})
    )
    long_sample['number_of_visitors'] = long_sample['number_of_visitors'].astype(int)
    long_sample['visitor_home_tract'] = long_sample['visitor_home_tract'].astype(str)
    long_sample['visitor_home_tract'] = long_sample['visitor_home_tract'].str.zfill(11)
    long_sample['poi_cbg'] = long_sample['poi_cbg'].astype(str)
    long_sample['poi_cbg'] = long_sample['poi_cbg'].str.zfill(12)
    parsed_df.append(long_sample)
  parsed_df = pd.concat(parsed_df)

  # Truncate values in both CBG columns to 5 digits (county)
  parsed_df['visitor_county_FIPS'] = parsed_df['visitor_home_tract'].str[:5]
  parsed_df['poi_county_FIPS'] = parsed_df['poi_cbg'].str[:5]
  parsed_df = parsed_df.drop(['visitor_home_tract', 'poi_cbg'], axis=1)

  # Aggregate rows with the same origin and destination state FIPS
  agg_state = parsed_df.groupby(['visitor_county_FIPS', 'poi_county_FIPS']).agg({'number_of_visitors': 'sum'})
  agg_state.columns = ['agg_visits']
  agg_state = agg_state.reset_index()
  with open(output_folder + filename_in[66:79] + ".csv", 'w', newline='') as f:
    writer = csv.writer(f)
    for i in range(len(agg_state)):
      writer.writerow([agg_state["visitor_county_FIPS"][i], agg_state["poi_county_FIPS"][i], agg_state["agg_visits"][i]])
    
  print("aggregation finished for: " + filename_in)

#Data Processing

In [ ]:
aggregate_data(filenames_in[2])

aggregate for file: /content/gdrive/MyDrive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part1.csv.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 1169/1169 [1:31:18<00:00,  4.69s/it]


aggregation finished for: /content/gdrive/MyDrive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part1.csv.gz


In [ ]:
filenames_in[0][65:78]

'2021-08-part1'

In [ ]:
aggregate_data(filenames_in[4])
aggregate_data(filenames_in[6])
aggregate_data(filenames_in[8])
aggregate_data(filenames_in[10])

NameError: ignored

In [17]:
aggregate_data(filenames_in[5])
aggregate_data(filenames_in[7])
aggregate_data(filenames_in[9])
aggregate_data(filenames_in[11])

aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part2.csv.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 993/993 [54:13<00:00,  3.28s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part2.csv.gz
aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part4.csv.gz


100%|██████████| 1206/1206 [1:08:45<00:00,  3.42s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-09-part4.csv.gz
aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part2.csv.gz


100%|██████████| 893/893 [49:28<00:00,  3.32s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part2.csv.gz
aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part4.csv.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 1266/1266 [1:06:19<00:00,  3.14s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part4.csv.gz


In [18]:
aggregate_data(filenames_in[1])
aggregate_data(filenames_in[0])

aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part2.csv.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
100%|██████████| 974/974 [50:46<00:00,  3.13s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part2.csv.gz
aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-08-part1.csv.gz


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2828: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
  0%|          | 3/1169 [00:14<1:32:09,  4.74s/it]


KeyboardInterrupt: ignored

In [19]:
aggregate_data(filenames_in[8])

aggregate for file: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part1.csv.gz


100%|██████████| 1267/1267 [1:08:04<00:00,  3.22s/it]


aggregation finished for: /content/gdrive/My Drive/COVID_DGNN_KG/Data/Mobility_raw/patterns-2021-10-part1.csv.gz
